In [1]:
# Import Libraries from Pyspark
import pyspark
import re
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
from datetime import datetime
import os
from IPython.display import Image
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.functions import sha2
from pyspark.sql import SQLContext

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('pro_processing_level_1').getOrCreate()

In [7]:
#file locations
raw_data_uri='/home/jovyan/work/hashed_data.csv'
page_mapping_uri='/home/jovyan/work/pagename_category.csv'
component_mapping_uri='/home/jovyan/work/component_mapping.csv'

In [8]:
#load data into spark and read files
df = spark.read.csv(raw_data_uri,sep=',',encoding='ISO-8859-1',header=True) 
#read the page category mapping file
df_pagename_mapping = spark.read.csv(page_mapping_uri, header=True)
#read the component mapping file
df_compname_mapping = spark.read.csv(component_mapping_uri, header=True)

In [9]:
#create dictionary to map component and page category
dict_compename_mapping = {row['Component']:row['Component Category'] for row in df_compname_mapping.select('Component','Component Category').collect()}
dict_pagename_mapping = {row['PageName']:row['Category'] for row in df_pagename_mapping.select('PageName','Category').collect()}


In [10]:
#map the pagename category & Component
def udf_wrapper(returntype):
        def udf_func(func):
            return udf(func, returnType=returntype)
        return udf_func
        
@udf_wrapper(StringType())
def get_comp_category(comp_type):
    '''
    based on the page_type return page category
    '''
    if comp_type is None:
        return None
    if comp_type == 'null':
        return None
    if comp_type in dict_compename_mapping.keys():
        return dict_compename_mapping[comp_type]
    else:
        return None
    
@udf_wrapper(StringType())
def get_page_category(page_type):
    '''
    based on the page_type return page category
    '''
    if page_type is None:
        return None
    if page_type == 'null':
        return None
    if page_type in dict_pagename_mapping.keys():
        return dict_pagename_mapping[page_type]
    else:
        return None

In [11]:
#create a new colum with component category and page category
df=df.withColumn('comp_category', get_comp_category(col('evar26')))
df=df.withColumn('page_category', get_page_category(col('post_pagename')))

In [12]:
#remove records which doesnt capture pagename and component
df_processed = df.filter(col('hit_source') == '1') \
                    .filter(col('exclude_hit') == '0')\
                    .filter(~((df.page_category.isNull()) & (df.comp_category.isNull())))

In [13]:
df_processed.count()

1640198

In [14]:
#Removing outliers
df_agg=df_processed.groupBy('hashed_visitor_id').count().orderBy('count',ascending=False)

In [15]:
#filter outliers with high or less number of activities 
df_filter=df_agg.filter(df_agg['count'] < 10000).filter(df_agg['count'] > 2)

In [16]:
#combine with original dataframe 
df_processed = df_processed.join(df_filter, on='hashed_visitor_id', how='inner')

In [17]:
df_processed.count()

1007178

In [18]:
# sessionization
TIME_OUT = 30 # 30 minutes of timeout


@udf_wrapper(IntegerType())
def get_event_boundary(time_diff):
    '''
    based on the timeout value, return the event boundary
    '''
    if(time_diff is None):
        return 0
    else:
        if(time_diff > TIME_OUT):
            return 1
        else:
            return 0

@udf_wrapper(FloatType())
def set_default_page_time(time_spent):
    '''
    based on the time spent,
    '''
    if(time_spent is None):
        return 0.08 # 5 seconds 
    else:
        return time_spent

In [19]:
def conversion_test_drive(page_category):
        if page_category == 'test_drive':
            return 1
        else : 
            return 0
    
conversion_test_drive_udf = udf(conversion_test_drive, IntegerType())

In [20]:
def conversion_cc(page_category):
        if page_category == 'your_vehicle':
            return 1
        else : 
            return 0
    
conversion_cc_udf = udf(conversion_cc, IntegerType())

In [21]:
# Timestamp Conversion
ts_pattern_1 = 'yyyy-MM-dd HH:mm:ss'
date_pattern_1 = 'yyyy-MM-dd'

In [22]:
df_processed = df_processed\
                     .withColumn('date_time_ts',unix_timestamp(col('date_time'), ts_pattern_1).cast('timestamp'))\
                     .withColumn('conversion_status_td', conversion_test_drive_udf('page_category'))\
                     .withColumn('conversion_status_cc', conversion_cc_udf('page_category'))
                     
    
                         
    
# Sessionization
w0 = Window.partitionBy(col('hashed_visitor_id'))

w1 = Window.partitionBy(col('hashed_visitor_id')).orderBy(col('date_time_ts'))

w2 = Window.partitionBy(col('hashed_visitor_id'),col('session_id')).orderBy(col('date_time_ts').desc())

w3 = Window.partitionBy(col('hashed_visitor_id'),col('session_id')).orderBy(col('date_time_ts'))

df_processed  = df_processed \
                   .withColumn('is_converted_td', max(col('conversion_status_td')).over(w0)) \
                   .withColumn('is_converted_cc', max(col('conversion_status_cc')).over(w0)) \
                   .withColumn('prev', lag(col('date_time_ts'),1).over(w1)) \
                   .withColumn('time_diff',((col('date_time_ts').cast('long') - col('prev').cast('long'))/60.0)) \
                   .withColumn('new_event_boundary',get_event_boundary(col('time_diff'))) \
                   .withColumn('session_id', sum(col('new_event_boundary')).over(w1)) \
                   .drop('prev','time_diff','new_event_boundary')

                    
# hit order
df_processed = df_processed.withColumn('hit_rank_reversed', dense_rank().over(w2))



#Time Spent                      
df_processed = df_processed\
                    .withColumn('next', lead(col('date_time_ts'),1).over(w3)) \
                    .withColumn('time_diff',((col('next').cast('long') - col('date_time_ts').cast('long'))/60.0)) \
                    .withColumn('time_spent_in_mins',set_default_page_time(col('time_diff'))) 


In [23]:
df_processed.count()

1007178

In [24]:
df_final=df_processed.select(['hashed_visitor_id','page_category','comp_category','time_spent_in_mins','session_id','is_converted_td','is_converted_cc',])

In [26]:
df_final.select('comp_category').distinct().show(50,False)

+----------------------------------------------------+
|comp_category                                       |
+----------------------------------------------------+
|Left_Rail_Panel                                     |
|NBA_palette_RequestTestDrive                        |
|Video_100%                                          |
|Configurator                                        |
|Sub_Navigation_Configure                            |
|Compare_Motors                                      |
|Configurator_Package                                |
|Vehicle_Thumbnail_                                  |
|Global_Navigation-Flyout_Menu                       |
|null                                                |
|Right_Rail_Action_Panel_DealerSearch                |
|Close_Configurator_Interior_Exterior                |
|Close_Configurator_Upholstery                       |
|Notifications                                       |
|Vehicle_Feature_Gallery__Expand                     |
|Right_Rai

In [27]:
df_comp=df_final.filter(col('comp_category') != '')

In [36]:
df_comp_category_counts = df_comp.groupby('hashed_visitor_id').pivot('comp_category').agg(count('hashed_visitor_id').cast('float'))


In [37]:
excluded_comp_category = ['Configurator_Summary','Others','others']

In [38]:
filtered_columns = ([column for column in df_comp_category_counts.columns if column not in excluded_comp_category])
df_comp_category_counts = df_comp_category_counts.select(filtered_columns)
df_comp_category_counts = df_comp_category_counts.fillna(0.0)

In [39]:
replacements = {c:'{0}'.format(c) for c in df_comp_category_counts.columns if c != 'hashed_visitor_id'}

In [40]:
df_comp_category_counts = df_comp_category_counts.select([col(c).alias(replacements.get(c, c)) for c in df_comp_category_counts.columns])


In [41]:
df_comp_category_counts.columns

['hashed_visitor_id',
 'Close_Configurator',
 'Close_Configurator_Accessories',
 'Close_Configurator_Interior_Exterior',
 'Close_Configurator_Packages',
 'Close_Configurator_Summary_-_CTA',
 'Close_Configurator_Upholstery',
 'Close_Content_Gallery_-_Inspiration_Video',
 'Compare_Motors',
 'Configurator',
 'Configurator_Package',
 'Configurator_Summary_-_CTA',
 'Content_Gallery_-_Equipment',
 'Content_Gallery_-_Inspiration',
 'Content_Gallery_-_Inspiration_-_IT',
 'Content_Gallery_-_Inspiration_Video',
 'Error_Message',
 'Expand_Car_Listing_AMG',
 'Finance_Calculator',
 'Footer',
 'Footer_RequestTestDrive',
 'Global_Navigation',
 'Global_Navigation-Flyout_Menu',
 'Key_Action_Tray_Model_Explore_page',
 'Left_Rail_Panel',
 'NBA_palette',
 'NBA_palette_RequestTestDrive',
 'Notifications',
 'Right_Rail_Action_Panel',
 'Right_Rail_Action_Panel_ConfigureOriginalAccessories',
 'Right_Rail_Action_Panel_ContactUs',
 'Right_Rail_Action_Panel_DealerSearch',
 'Right_Rail_Action_Panel_DiscoverOffer'

In [42]:
aggregations = []
aggregations.append(max(col('is_converted_cc')).alias('is_converted_visitor'))

In [43]:
df_aggregated_others = df_comp.select('hashed_visitor_id','is_converted_cc').groupBy('hashed_visitor_id').agg(*aggregations)


In [44]:
df_final =  df_comp_category_counts.join(df_aggregated_others,'hashed_visitor_id', 'inner')

In [45]:
model_inputs=[col for col in df_final.columns if col not in ['hashed_visitor_id','is_converted_visitor']]

In [46]:
#creating the assemblerfor input variables 
assembler=VectorAssembler(inputCols=model_inputs,outputCol='features')


In [47]:
#creating dense vector represntation of input variables
output=assembler.transform(df_final)



In [48]:
#declaring the input dense vector and output variable
data=output.select('features','is_converted_visitor')


In [49]:
#build and train the ML model
rfc=RandomForestClassifier(labelCol='is_converted_visitor',featuresCol='features')

In [50]:
#fit the model on training data 
rf_model=rfc.fit(data)

In [51]:
#map the comp ids back to components
feats={}
for feature,importance in zip(model_inputs,rf_model.featureImportances):
    feats[feature]=importance

In [52]:
feats.keys()

dict_keys(['Close_Configurator', 'Close_Configurator_Accessories', 'Close_Configurator_Interior_Exterior', 'Close_Configurator_Packages', 'Close_Configurator_Summary_-_CTA', 'Close_Configurator_Upholstery', 'Close_Content_Gallery_-_Inspiration_Video', 'Compare_Motors', 'Configurator', 'Configurator_Package', 'Configurator_Summary_-_CTA', 'Content_Gallery_-_Equipment', 'Content_Gallery_-_Inspiration', 'Content_Gallery_-_Inspiration_-_IT', 'Content_Gallery_-_Inspiration_Video', 'Error_Message', 'Expand_Car_Listing_AMG', 'Finance_Calculator', 'Footer', 'Footer_RequestTestDrive', 'Global_Navigation', 'Global_Navigation-Flyout_Menu', 'Key_Action_Tray_Model_Explore_page', 'Left_Rail_Panel', 'NBA_palette', 'NBA_palette_RequestTestDrive', 'Notifications', 'Right_Rail_Action_Panel', 'Right_Rail_Action_Panel_ConfigureOriginalAccessories', 'Right_Rail_Action_Panel_ContactUs', 'Right_Rail_Action_Panel_DealerSearch', 'Right_Rail_Action_Panel_DiscoverOffer', 'Right_Rail_Action_Panel_FB', 'Right_Rail

In [53]:
feats.values()

dict_values([0.089784994516350261, 0.051577013672406594, 0.00012984245479771931, 0.051893591713808737, 0.010491188255594693, 0.028042334233001072, 0.0041197609997005837, 0.015997359030669487, 0.22230139902807369, 0.0, 0.48499886956611293, 0.00033236793620099907, 0.00046205716362213069, 3.0259220792885212e-05, 0.00071216266839069621, 0.0, 0.00010583161944851024, 0.0, 0.00044472865059256477, 0.00015673650209578231, 0.00878635320557139, 0.00010886997513282176, 0.0, 0.00043843291972471928, 4.5407959170823869e-05, 0.0, 0.012461834781056728, 5.2144742517002955e-05, 2.7946444029010856e-05, 9.7538662127331615e-05, 0.0, 0.0, 0.0, 3.4906457814097404e-05, 0.0041380676651072128, 0.00060120663571228009, 0.0019352155915533815, 8.8502030444424692e-05, 7.5995736545309071e-05, 0.0066513213219370915, 0.0, 8.0969033447990569e-05, 0.0007939230205057324, 0.00060860869363292135, 0.0006736710474370875, 0.00051664746851993451, 1.5919376479893821e-05, 1.1027348503484078e-05, 1.7462261496855768e-05, 0.000133196

In [54]:
results=sorted(feats.items(),key=lambda x:x[1],reverse=True)

In [55]:
final_results_comp=[(key,str(imp)) for key,imp in results]
results_df = spark.createDataFrame(final_results_comp, ['key','val'])

In [56]:
results_df.show(50,False)

+----------------------------------------------------+-----------------+
|key                                                 |val              |
+----------------------------------------------------+-----------------+
|Configurator_Summary_-_CTA                          |0.484998869566   |
|Configurator                                        |0.222301399028   |
|Close_Configurator                                  |0.0897849945164  |
|Close_Configurator_Packages                         |0.0518935917138  |
|Close_Configurator_Accessories                      |0.0515770136724  |
|Close_Configurator_Upholstery                       |0.028042334233   |
|Compare_Motors                                      |0.0159973590307  |
|Notifications                                       |0.0124618347811  |
|Close_Configurator_Summary_-_CTA                    |0.0104911882556  |
|Global_Navigation                                   |0.00878635320557 |
|Vehicle_Feature_Gallery                           